In [13]:
#Importing all modules

import importlib
import scripts_raw_data.fetch_raw_data as r
importlib.reload(r)

<module 'scripts_raw_data.fetch_raw_data' from 'c:\\Users\\eloib\\Perso\\Fac\\M1\\HoML\\Projet-Homl\\scripts_raw_data\\fetch_raw_data.py'>

# I. Fetching the raw data.


In [15]:
r.download_season_stats(1980, 2025)
r.download_expanded_standings(1980, 2025)
print("[INFO] All requested downloads finished.")


[INFO] Downloading per-season raw data
[INFO] Downloading season 1980 from https://www.basketball-reference.com/leagues/NBA_1980_per_game.html...
[SKIP] Skipping season 1980, file already exists.
[INFO] Downloading season 1981 from https://www.basketball-reference.com/leagues/NBA_1981_per_game.html...
[SKIP] Skipping season 1981, file already exists.
[INFO] Downloading season 1982 from https://www.basketball-reference.com/leagues/NBA_1982_per_game.html...
[SKIP] Skipping season 1982, file already exists.
[INFO] Downloading season 1983 from https://www.basketball-reference.com/leagues/NBA_1983_per_game.html...
[SKIP] Skipping season 1983, file already exists.
[INFO] Downloading season 1984 from https://www.basketball-reference.com/leagues/NBA_1984_per_game.html...
[SKIP] Skipping season 1984, file already exists.
[INFO] Downloading season 1985 from https://www.basketball-reference.com/leagues/NBA_1985_per_game.html...
[SKIP] Skipping season 1985, file already exists.
[INFO] Downloading